<a href="https://colab.research.google.com/github/gabibu/nlp/blob/master/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


*As a preparation for this task, download the data files from the course git repository.

The relevant files are under **lm-languages-data-new**:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)
*   test.csv (or the equivalent JSON file)





In [1]:
!git clone https://github.com/kfirbar/nlp-course.git

Cloning into 'nlp-course'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 53 (delta 23), reused 32 (delta 8), pack-reused 0
Unpacking objects: 100% (53/53), done.




---



**Important note: please use only the files under lm-languages-data-new and NOT under lm-languages-data**


---



In [2]:

!ls nlp-course/lm-languages-data-new


en.csv	 es.json  in.csv   it.json  pt.csv    test.json   tl.csv
en.json  fr.csv   in.json  nl.csv   pt.json   tests.csv   tl.json
es.csv	 fr.json  it.csv   nl.json  test.csv  tests.json


**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [3]:
import os
import io
import pathlib
import pandas as pd
pd.set_option("max_colwidth", 140)
import json 

folder = 'nlp-course/lm-languages-data-new'

def read_tweets(file_path):
  file_suffix = pathlib.Path(file_path).suffix
  if file_suffix == '.json':
    with open(file_path) as f:
      data = json.load(f)
      return [tweet.lower() for tweet in data['tweet_text'].values()]
      
        
  elif file_suffix == '.csv':
    df = pd.read_csv(file_path, header=0)
    return [tweet.lower() for tweet in df['tweet_text'].tolist()]

def preprocess():

  vocab = set()
  for file_name in os.listdir(folder):
    file_path = os.path.join(folder, file_name)
    tweets = read_tweets(file_path)

    vocab.update({word for tweet in tweets for word in tweet.lower()})

  return vocab
        

**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [7]:
from collections import defaultdict
import numpy as np 

start = '<S>'
end = '<E>'

def build_probs_model(n, vocabulary, tweets, add_one):

    counts = {}

    num_of_tokens = 0.0
    for tweet in tweets:
      
        characters = list(tweet)

        tokens = [start] + characters + [end] if n > 1 else characters

        num_of_tokens += len(tokens)

        if len(tokens) < n:
            continue

        for i in range(n, len(tokens)):
            current_tokens = tokens[i - n: i]

            character = current_tokens[-1]

            if n > 1:
                n_key = ''.join(current_tokens[0: -1])

                if n_key not in counts:
                    counts[n_key] = {}

                if character not in counts[n_key]:
                    counts[n_key][character] = 0.0

                counts[n_key][character] += 1.0
            else:
                if character not in counts:
                    counts[character] = 0.0

                counts[character] += 1


    if n == 1:
        
        if add_one:
            model = defaultdict(lambda : 1.0/len(vocabulary), {word: (count + 1.0) / (num_of_tokens + len(vocabulary)) for (word, count) in counts.items()})
        else:
          model = defaultdict(lambda : 0.0, {word: count / num_of_tokens for (word, count) in counts.items()})
    else:
            
        default_model = defaultdict(lambda :0.0) if not add_one else defaultdict(lambda :1.0/len(vocabulary))
        model = defaultdict(lambda : default_model)
        for gram, next_character_counts in counts.items():
            n = np.sum(list(next_character_counts.values()))

            if add_one:
              model[gram] = defaultdict(lambda: 1.0/(n + len(vocabulary)), {word: (count +1.0) / (n + len(vocabulary)) for (word, count) in next_character_counts.items()})
            else:
              model[gram] = defaultdict(lambda : 0.0, {word: count / n for (word, count) in next_character_counts.items()})

    return model






def lm(n, vocabulary, data_file_path, add_one):
  # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
  # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
  # data_file_path - the data_file from which we record probabilities for our model
  # add_one - True/False (use add_one smoothing or not)

  # TODO
  tweets = read_tweets(data_file_path)
  return build_probs_model(n, vocabulary, tweets, add_one)

**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.

In [8]:
unk_prob = 0.000000001
def eval(n, model, data_file):
    # n - the n-gram that you used to build your model (must be the same number)
    # model - the dictionary (model) to use for calculating perplexity
    # data_file - the tweets file that you wish to claculate a perplexity score for

    # TODO
    tweets = read_tweets(data_file)

    tweets_perplexity = []
    for tweet in tweets:
        probs_n = 0.0
        entropy_sum = 0.0
        if n == 1:
            for token in tweet:
                prob = model[token]
                
                entropy_sum += unk_prob * np.log2(unk_prob) if prob == 0 else prob * np.log2(prob)
                probs_n += 1
        else:
            for index in range(0, len(tweet)):
                to_index = index + n
                tokens = tweet[index: to_index]
                if len(tokens) < n:
                    break
                prev = tokens[0: -1]
                to_token = tokens[-1]
                prob = model[prev][to_token]
                entropy_sum += unk_prob * np.log2(unk_prob) if prob == 0 else prob * np.log2(prob)
               
                probs_n += 1

        entropy = (-1 / probs_n) * entropy_sum
        tweet_perplexity = np.power(2, entropy)
        tweets_perplexity.append(tweet_perplexity)

    return np.mean(tweets_perplexity)

**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [9]:
def match(n, add_one):
  # n - the n-gram to use for creating n-gram models
  # add_one - use add_one smoothing or not

  vocabulary = preprocess()

  language_to_model = {}
  language_to_data_file = {}
  for file_name in os.listdir(folder):
    language = os.path.splitext(file_name)[0]
    if language in language_to_model or 'test' in language:
      continue

    file_path = os.path.join(folder, file_name)
    language_to_data_file[language] = file_path

    model = lm(n, vocabulary, file_path, add_one)
    language_to_model[language] = model

  languages = sorted(list(language_to_model.keys()))

  rows = []
  for index, language1 in enumerate(languages):
    current_res = []
    for index, language2 in enumerate(languages):
      perplexity = eval(n, language_to_model[language1], language_to_data_file[language2])
      current_res.append(perplexity)
    
    current_res.append(n)
    current_res.append(add_one)
    
    rows.append(current_res)
  
  return pd.DataFrame(rows, columns =languages + ["n", "add_one"], index=languages)

**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [10]:
frames = []
for n in range(1,5):
  for add_one in [True, False]:
    frames.append(match(n, add_one))
    
all_frames = pd.concat(frames)
all_frames

,en,es,fr,in,it,nl,pt,tl,n,add_one
en,1.151613,1.153592,1.150387,1.144434,1.150451,1.147355,1.152866,1.146226,1,True
es,1.150314,1.159726,1.153757,1.146553,1.154292,1.147915,1.158623,1.148209,1,True
fr,1.148823,1.155337,1.153364,1.142092,1.150591,1.146594,1.154142,1.141728,1,True
in,1.144004,1.149176,1.143402,1.153630,1.146235,1.141741,1.149750,1.154152,1,True
it,1.149675,1.156892,1.151586,1.146016,1.155480,1.146792,1.156340,1.147953,1,True
...,...,...,...,...,...,...,...,...,...,...
in,1.162569,1.141125,1.132623,1.238862,1.145076,1.126326,1.136304,1.168208,4,False
it,1.149051,1.165271,1.140862,1.114180,1.236818,1.113411,1.156911,1.136072,4,False
nl,1.167539,1.139856,1.143432,1.126287,1.143758,1.233318,1.132736,1.141531,4,False
pt,1.134598,1.168281,1.128880,1.105841,1.155399,1.103840,1.235598,1.130311,4,False


**Part 6**

Each line in the file test.csv contains a sentence and the language it belongs to. Write a function that uses your language models to classify the correct language of each sentence.

Important note regarding the grading of this section: this is an open question, where a different solution will yield different accuracy scores. any solution that is not trivial (e.g. returning 'en' in all cases) will be excepted. We do reserve the right to give bonus points to exceptionally good/creative solutions.

In [11]:
def build_languages_models(n, add_one):

  vocabulary = preprocess()

  language_to_model = {}
  for file_name in os.listdir(folder):
    language = os.path.splitext(file_name)[0]
    if language in language_to_model or 'test' in language:
      continue
      
    
    file_path = os.path.join(folder, file_name)

    model = lm(n, vocabulary, file_path, add_one)
    language_to_model[language] = model
  return language_to_model

two_gram_language_to_model = build_languages_models(2, True)
unigram_gram_language_to_model = build_languages_models(1, True)

In [12]:
TEST_FILE = 'test.csv'
test_df = pd.read_csv(os.path.join(folder, TEST_FILE))[['tweet_text', 'label']]
test_df["tweet_text"] = test_df["tweet_text"].str.lower()

In [26]:
import string
import re

def clean_text(tweet):

  if tweet.startswith('rt'):
    tweet = tweet.replace('rt', '')

  seen = False
  
  xx = ''
  for x in tweet:
    if seen:
      if x == ' ':
        seen = False
      else:
        continue
    else:
     
      if x in ['😂', '😓', '👌🏽','🕋','💎','😌', '🆘', '🔴','#', '😍','🔥','💙','😻','💔', '🙌🏾','👙','☀️','🍾','🎈','%', '😈','💦','!', '🤘🏼',
               '0','1','2','3','4','5','6','7','8','9', '💨', '🙋🏻', '❤', '😆', '😲', '🤦‍♀️']:
        xx +=''
        continue

      if x == '@':
        seen = True
      else:
        xx += x

  g = re.findall(r'(https?://\S+)', xx)

  for x in g:
    xx = xx.replace(x, '')
  
  xx = xx.replace(' ', ' ')
  return xx.strip()


In [35]:

test_df['clean_tweet_text'] = test_df['tweet_text'].apply(clean_text)

In [44]:

def classify_language(tweet, n):
  
  best_language_prob = float('-inf')
  best_language = None

  for language, model in two_gram_language_to_model.items():
    if language in {'en', 'nl'}:
      print('lan: {}'.format(language))
    unigram_model = unigram_gram_language_to_model[language]

    index = 0

    language_log_prob = 0.0
    while index + n <= len(tweet):

      if n == 1:
        token = tweet[index: index + 1]
        prob = model[token]
        prob = prob if prob > 0 else 0.0000001
        language_log_prob += np.log(prob)
      else:
        tokens = tweet[index: index + n]
        prev = tokens[0:-1]
        next = tokens[-1]

        prob = model[prev][next]
        if language in {'en', 'nl'}:
          print('tokens {} prob {}'.format(tokens, prob))
        prob = prob if prob > 0 else 0.0000001
        uni_prob = unigram_model[next] if unigram_model[next] > 0 else 0.0000001
        language_log_prob += np.log((0.8 * prob)+ (uni_prob * 0.2))
      index += 1
    
   
    if language_log_prob > best_language_prob:
      best_language_prob = language_log_prob
      best_language = language
    

  return best_language

In [57]:

correct_df = test_df[test_df['label'] == test_df['predicted_label']]



correct_size_df = correct_df.groupby('label').size().reset_index()
correct_size_df.sort_values(by= 0, ascending=False)

,label,0
2,fr,925
0,en,901
1,es,898
6,pt,893
3,in,870
4,it,865
5,nl,852
7,tl,782


In [55]:

missed_df = test_df[test_df['label'] != test_df['predicted_label']]

missed_df['pair'] = missed_df.apply(lambda r: (r['predicted_label'], r['label']), axis=1)

missed__size_df = missed_df.groupby('pair').size().reset_index()
missed__size_df.sort_values(by= 0, ascending=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,pair,0
41,"(nl, tl)",80
27,"(in, tl)",78
4,"(en, nl)",56
12,"(es, pt)",54
52,"(tl, in)",47
6,"(en, tl)",39
10,"(es, it)",35
2,"(en, in)",34
43,"(pt, es)",31
54,"(tl, nl)",31


In [40]:
test_df["predicted_label"] = test_df["clean_tweet_text"].apply(lambda t: classify_language(t, 2))

In [41]:
len(test_df[test_df['label'] == test_df['predicted_label']])/len(test_df)

0.8733591698962371

In [42]:
test_df[test_df['label'] != test_df['predicted_label']].sample(3)[['clean_tweet_text', 'tweet_text', 'predicted_label', 'label']]

,clean_tweet_text,tweet_text,predicted_label,label
4375,no sirvo ni para hacer arroz🤦‍♀️,no sirvo ni para hacer arroz🤦‍♀️,it,es
4583,beep beep,@clemstaldim beep beep https://t.co/sxcnwsxun8,nl,en
2175,trop,@godshesacrybaby trop 😂😂,nl,fr


**Part 7**

Calculate the F1 score of your output from part 6. (hint: you can use https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html). 


In [43]:
from sklearn.metrics import f1_score

f1_score(test_df['label'].tolist(), test_df['predicted_label'].tolist(), average='micro')

0.873359169896237

# **Good luck!**